# Multi-Linear Face Model

This Jupyter notebook playes around with a MLM motivated by [Vlasic et al](https://vcg.seas.harvard.edu/publications/face-transfer-with-multilinear-models/paper).

The MLM decomposes a face tensor using a Tucker Decomposition, which can be described as a three-mode PCA and can be generalized to higher orders analysis as HO-SVD.

We show how a MLM model can be used to fill missing Data, interpolate between expressions and identites, as well as fit new identities and expressions. Compared to a PCA model a MLM model allows more precise and separate control over identity and expressions.

In [1]:
import os
import meshio
import numpy as np
import matplotlib.pyplot as plt

from model import MLM
from helpers import *
import torch
import tensorly as tl
import trimesh
import pyrender

import tensorly as tl
#import redner # differentiable renderer

In [2]:
# Tensorly can be used with different backends, we used pyTorch
tl.set_backend("pytorch")

## Expression and Identity

Given a MLM model we can play around with the factors for expression and identity and create new or interpolated identities as well as expressions. Unfortunately our Dataset has not a very large variatey of expressions. More drastic changes like and open mouth expression would allow a more interesting analysis of the expression space.

In [3]:
# Load a preceomputed MLM without any compression
mlm = MLM()
mlm.load("model/precomputeReconstruction")
print("Expression_knobs: ", mlm.factor_exp.shape[1])
print("Identity_knobs: ", mlm.factor_id.shape[1])

Multilinear Model
load...
done!
Expression_knobs:  2
Identity_knobs:  41


/Users/Agon/Documents/Programming/ETH_Programming/SMGP/repo/smgp/BonusMLM/model/MLM.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.core = torch.tensor(torch.load(os.path.join(path, "core.pt")))
/Users/Agon/Documents/Programming/ETH_Programming/SMGP/repo/smgp/BonusMLM/model/MLM.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.factor_exp = torch.tensor(torch.load(os.path.join(path, "factor_exp.pt")))
/Users/Agon/Documents/Programming/ETH_Programming/SMGP/repo/smgp/BonusMLM/model/MLM.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(so

### Similar to our PCA morphing task we can interpolate between two identities

In [4]:
interpolation = 0.5 # set a value in [0..1]
identity_1 = 1
identity_2 = 13

exp_vector = mlm.factor_exp[0:1,:] # neutral expression code

# mix
id_vector = (interpolation * mlm.factor_id[identity_1:identity_1+1,:]) + ((1.0-interpolation) * mlm.factor_id[identity_2:identity_2+1,:]) # morphed identity
face = unflat_mesh(mlm.reconstruct(exp_vector, id_vector))
mesh = trimesh.Trimesh(vertices=face, faces=mlm.indices)

# identity 1
id_vector1 = mlm.factor_id[identity_1:identity_1+1,:]
face1 = unflat_mesh(mlm.reconstruct(exp_vector, id_vector1))
mesh1 = trimesh.Trimesh(vertices=face1, faces=mlm.indices)

# identity 2
id_vector2 = mlm.factor_id[identity_2:identity_2+1,:]
face2 = unflat_mesh(mlm.reconstruct(exp_vector, id_vector2))
mesh2 = trimesh.Trimesh(vertices=face2, faces=mlm.indices)



save_mesh(mesh1, "identity_morphing_id1.obj")
save_mesh(mesh2, "identity_morphing_id2.obj")
save_mesh(mesh, "identity_morphing_mix.obj")
mesh.show()

In [6]:
# reconstruct a whole morphing sequence
count = 0
for i in [i*0.01 for i in range(101) ]:
    identity_1 = 1
    identity_2 = 13

    exp_vector = mlm.factor_exp[0:1,:] # neutral expression code

    # mix
    id_vector = (i * mlm.factor_id[identity_1:identity_1+1,:]) + ((1.0-i) * mlm.factor_id[identity_2:identity_2+1,:]) # morphed identity
    face = unflat_mesh(mlm.reconstruct(exp_vector, id_vector))
    mesh = trimesh.Trimesh(vertices=face, faces=mlm.indices)

    save_mesh(mesh, "identity_morphing_mix_{}.obj".format(count))
    print(i)
    count += 1


0.0
0.01
0.02
0.03
0.04
0.05
0.06
0.07
0.08
0.09
0.1
0.11
0.12
0.13
0.14
0.15
0.16
0.17
0.18
0.19
0.2
0.21
0.22
0.23
0.24
0.25
0.26
0.27
0.28
0.29
0.3
0.31
0.32
0.33
0.34
0.35000000000000003
0.36
0.37
0.38
0.39
0.4
0.41000000000000003
0.42
0.43
0.44
0.45
0.46
0.47000000000000003
0.48
0.49
0.5
0.51
0.52
0.53
0.54
0.55
0.56
0.5700000000000001
0.58
0.59
0.6
0.61
0.62
0.63
0.64
0.65
0.66
0.67
0.68
0.6900000000000001
0.7000000000000001
0.71
0.72
0.73
0.74
0.75
0.76
0.77
0.78
0.79
0.8
0.81
0.8200000000000001
0.8300000000000001
0.84
0.85
0.86
0.87
0.88
0.89
0.9
0.91
0.92
0.93
0.9400000000000001
0.9500000000000001
0.96
0.97
0.98
0.99
1.0


## But we can also create new Identities and let them smile :) 

In [40]:
# generate meshes with the help of the identity vectors of the others. 
# This is not necessary but makes it simpler to get the scale of the identity vector right
for count in range(100):
    strenght = 1
    mix1 = np.random.randint(0, high=mlm.factor_id.shape[0])
    id1 = mlm.factor_id[mix1:mix1+1,:]
    noise = torch.rand((1,mlm.factor_id.shape[1]))*strenght # Random Identity
    id1 = noise*id1 / torch.norm(noise)
    
    mix2 = np.random.randint(0, high=mlm.factor_id.shape[0])
    id2 = mlm.factor_id[mix2:mix2+1,:]
    noise = torch.rand((1,mlm.factor_id.shape[1]))*strenght # Random Identity
    id2 = noise*id2 / torch.norm(noise)
    
    mix3 = np.random.randint(0, high=mlm.factor_id.shape[0])
    id3 = mlm.factor_id[mix3:mix3+1,:]
    noise = torch.rand((1,mlm.factor_id.shape[1]))*strenght # Random Identity
    id3 = noise*id3 / torch.norm(noise)
    
    mix4 = np.random.randint(0, high=mlm.factor_id.shape[0])
    id4 = mlm.factor_id[mix4:mix4+1,:]
    noise = torch.rand((1,mlm.factor_id.shape[1]))*strenght # Random Identity
    id4 = noise*id4 / torch.norm(noise)
    
    id_vector = id1 * 0.25 + id2 * 0.25 + id3 * 0.25 + id4 * 0.25
    
    exp_vector = mlm.factor_exp[1:2,:] # smile expression code

    # compute mesh
    face = unflat_mesh(mlm.reconstruct(exp_vector, id_vector))
    mesh = trimesh.Trimesh(vertices=face, faces=mlm.indices)

    save_mesh(mesh, "random_smile_{}.obj".format(count))

mesh.show()

## Fill Missing Data

One possible application of MLM is to fill in missing Data. 
For this we removed the smiling expression of subject 5 from the Datatensor. Next we decompose the tensor and are able to approximate the missing expression. 

In [5]:
# Load the preceomputed MLM
mlm = MLM()
mlm.load("model/precomputeMissingData")
print(mlm.factor_exp.shape)
print(mlm.factor_id.shape)

Multilinear Model
load...
done!
torch.Size([2, 1])
torch.Size([41, 31])


In [7]:
groundtruth = "../allTasksExceptLandmarks/data/nonRigidFaces/fabian_smile_non_rigid_aligned.obj"
vert, _ = load_mesh(groundtruth)
meshGT = trimesh.Trimesh(vertices=unflat_mesh(vert), faces=mlm.indices)
meshGT.show()

/Users/Agon/opt/anaconda3/envs/SMGP/lib/python3.7/site-packages/tensorly/backend/pytorch_backend.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  requires_grad=requires_grad)


In [79]:
exp_vector = mlm.factor_exp[1:2,:] # expression code for smiling
id_vector = mlm.factor_id[9:10,:] # the subject from whom we are missing smiling-expression

face = unflat_mesh(mlm.reconstruct(exp_vector*3, id_vector))
mesh = trimesh.Trimesh(vertices=face, faces=mlm.indices)
mesh.show()